
## Building a Conversational Chatbot in your local with Llama 3.2 using Ollama

This notebook demonstrates a streamlined workflow for building a **Retrieval-Augmented Generation (RAG)** system locally using **Llama 3.2** via **Ollama**. The goal is to enable efficient document-based question answering by integrating document ingestion, vector database storage, and a conversational retrieval system.

The following steps are covered:

1. **Document Preparation and Splitting**:  PDF, CSV, and DOCX files from a specified directory are loaded, and the content is split into smaller, manageable chunks using the RecursiveCharacterTextSplitter. This process ensures optimal chunking based on size and overlap to improve retrieval accuracy during further processing.

2. **Ingesting Documents into Vector Database**: The split documents are embedded using `HuggingFaceEmbeddings` and stored in a local FAISS vector database, facilitating fast and scalable document retrieval.

3. **Building the Conversation Chain**: A conversational chain is built using **Llama 3.2** to retrieve relevant information based on user queries. This chain ensures that the responses are accurate, concise, and relevant to the context of the chat history, while managing session history to maintain the flow of conversation.

4. **Similarity Score Calculation**: To evaluate the relevancy of the system’s responses, the notebook includes a function to calculate the similarity score between the generated answers and the source documents using `SentenceTransformer`.

The notebook is designed for local execution, leveraging the performance and capabilities of Llama 3.2 via the **Ollama** API to offer a robust and efficient Q&A solution.

Below cell imports the required libraries to run this notebook.

In [21]:
import os
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [22]:
!pip install -r /content/drive/MyDrive/surya/FMGAI/requirements.txt


In [23]:
!pip install langchain-huggingface


In [24]:
from langchain.document_loaders import DirectoryLoader, CSVLoader, UnstructuredWordDocumentLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.llms import Ollama
from langchain.vectorstores import FAISS
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from sentence_transformers import SentenceTransformer, util
#from htmlTemplate import css, bot_template, user_template
from langchain_core.chat_history import BaseChatMessageHistory
from langchain.chains import create_history_aware_retriever
from langchain_huggingface import HuggingFaceEmbeddings

#### Specify values for the following:

- **`file_directory`**: Provide the name of the folder where your PDF,docx or csv files are stored.

- **`embedding_model`**: This is the model used to generate text embeddings. It will be applied to both chunked documents and for calculating the similarity score between the source documents and the LLM's response.

- **`llm_model`**: Refers to the language model being used,  "llama3.2" in this case.

In [26]:
file_directory="/content/drive/MyDrive/surya/FMGAI/data_directory"
embedding_model='sentence-transformers/all-MiniLM-L6-v2'
llm_model ="llama3.2"

### Step 1: Prepare documents and their metadata
This function, `prepare_and_split_docs()`, loads documents from a given directory, including PDFs, DOCX files, and CSVs, using `DirectoryLoader` with specific loaders for each file type. It then splits these documents into smaller chunks using a `RecursiveCharacterTextSplitter` with a defined chunk size and overlap. Finally, the function returns the split documents while maintaining metadata.

In [27]:
def prepare_and_split_docs(directory):
    # Load the documents
    loaders = [
        DirectoryLoader(directory, glob="**/*.pdf",show_progress=True, loader_cls=PyPDFLoader),
        DirectoryLoader(directory, glob="**/*.docx",show_progress=True),
        DirectoryLoader(directory, glob="**/*.csv",loader_cls=CSVLoader)
    ]


    documents=[]
    for loader in loaders:
        data =loader.load()
        documents.extend(data)

    # Initialize a text splitter
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512,  # Use the smaller chunk size here to avoid repeating splitting logic
        chunk_overlap=256,
        disallowed_special=(),
        separators=["\n\n", "\n", " "]
    )

    # Split the documents and keep metadata
    split_docs = splitter.split_documents(documents)

    print(f"Documents are split into {len(split_docs)} passages")
    return split_docs


### Step 3: Ingest into Vector Database locally

The `ingest_into_vectordb` function is designed for processing and indexing a collection of documents into a vector database using FAISS (Facebook AI Similarity Search) for efficient similarity searches. It operates as follows:

1. **Embedding Creation**: It generates embeddings for the input documents (`split_docs`) using the Hugging Face model `'sentence-transformers/all-MiniLM-L6-v2'`. This model is specifically chosen for its efficiency in creating sentence-level embeddings and is set to run on the CPU.

2. **Vector Database Indexing**: Utilizes the generated embeddings to create a FAISS vector database. FAISS is used for its ability to efficiently handle large-scale similarity searches and clustering of dense vectors.

3. **Local Storage**: After creating the vector database, the function saves it locally to the path specified by `DB_FAISS_PATH`, ensuring the data can be easily accessed for future similarity searches or retrieval tasks.

The primary purpose of this function is to transform textual data into a structured, searchable vector format, facilitating efficient and scalable retrieval tasks such as document similarity searches or clustering.

In [28]:
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
def ingest_into_vectordb(split_docs):
    db = FAISS.from_documents(split_docs, embeddings)

    DB_FAISS_PATH = 'vectorstore/db_faiss'
    db.save_local(DB_FAISS_PATH)
    print("Documents are inserted into FAISS vectorstore")
    return db

### Step 4: Set up Conversation Chain using LLM
The `get_conversation_chain(retriever)` function creates a stateful conversational RAG system.

1. It initializes the `llama3.2` model and defines two prompts:
   - A contextualization prompt to handle the user's query in light of the chat history.
   - A system prompt for answering concisely with 2-3 sentences based on retrieved documents.

2. It builds a `history_aware_retriever` using the retriever, LLM, and the contextualization prompt to ensure responses are context-aware.

3. A `question_answer_chain` is set up to respond with answers limited to 50 words.

4. These components are combined into a RAG chain using `create_retrieval_chain`.

5. To manage chat history across sessions, it defines `get_session_history`, which stores and retrieves message history by session ID.

6. Finally, a `RunnableWithMessageHistory` integrates the RAG chain with chat history management, ensuring the bot maintains state and provides contextually relevant responses throughout the conversation.

This function sets up a sophisticated conversational AI system combining the LLaMA model for language generation and a vector database for information retrieval, enhanced with a callback manager for additional processing and a conversation memory buffer for context management.

In [29]:
def get_conversation_chain(retriever):
    llm = Ollama(model=llm_model)
    contextualize_q_system_prompt = (
        "Given the chat history and the latest user question, "
        "provide a response that directly addresses the user's query based on the provided  documents. "
        "Do not rephrase the question or ask follow-up questions."
    )


    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )


    ### Answer question ###
    system_prompt = (
        "As a personal chat assistant, provide accurate and relevant information based on the provided document in 2-3 sentences. "
        "Answe should be limited to 50 words and 2-3 sentences.  do not prompt to select answers or do not formualate a stand alone question. do not ask questions in the response. "
        "{context}"
    )

    qa_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )
    question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

    rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


    ### Statefully manage chat history ###
    store = {}


    def get_session_history(session_id: str) -> BaseChatMessageHistory:
        if session_id not in store:
            store[session_id] = ChatMessageHistory()
        return store[session_id]


    conversational_rag_chain = RunnableWithMessageHistory(
        rag_chain,
        get_session_history,
        input_messages_key="input",
        history_messages_key="chat_history",
        output_messages_key="answer",
    )
    print("Conversational chain created")
    return conversational_rag_chain

### Step 5: Calculate Document Similarity in the LLMs Response
The `calculate_similarity_score` function computes the cosine similarity between a given answer and a list of context documents using Sentence Transformers. It first encodes the answer and context documents into embeddings. Then, it calculates the cosine similarities between the answer embedding and the context embeddings. The function returns the maximum similarity score, indicating how closely the answer relates to the most relevant context document. Scores range from 0 (no similarity) to 1 (perfect similarity), with higher scores reflecting better alignment with the context.

Essentially, this function serves as a mechanism to check the alignment of the chatbot's response with the information in the source documents, ensuring the response's accuracy and relevance.

In [30]:
def calculate_similarity_score(answer: str, context_docs: list) -> float:

    context_docs = [doc.page_content for doc in context_docs]

    # Encode the answer and context documents
    answer_embedding = embeddings.embed_query(answer)
    context_embeddings = embeddings.embed_documents(context_docs)

    # Calculate cosine similarities
    similarities = util.pytorch_cos_sim(answer_embedding, context_embeddings)

    # Return the maximum similarity score from the context documents
    max_score = similarities.max().item()
    return max_score


Now that we have crafted all the necessary functions, it's time to put them into action and test their functionality.

In [35]:
import nltk
nltk.download('punkt')
!pip install tiktoken


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.9 MB/s eta 0:00:00


In [36]:
split_docs=prepare_and_split_docs(file_directory)
vector_db= ingest_into_vectordb(split_docs)
retriever =vector_db.as_retriever()
conversational_rag_chain=get_conversation_chain(retriever)




100%|██████████| 1/1 [00:00<00:00, 72.95it/s]


0it [00:00, ?it/s]


Documents are split into 1 passages
Documents are inserted into FAISS vectorstore
Conversational chain created


### Ask your Question

We created a conversational chain and now ready to chat with your own data.


### Question 1

In [37]:
user_question="Can you summarise the documents ?"

In [43]:
qa1 = conversational_rag_chain.invoke(
    {"input": user_question},
    config={
        "configurable": {"session_id": "abc123"}
    }
)

# Check if 'answer' exists in the response
if "answer" in qa1:
    print(qa1["answer"])
else:
    print("No answer found in the response. Here's the raw response:")
    print(qa1)


ValueError: Missing keys ['session_id'] in config['configurable'] Expected keys are ['session_id'].When using via .invoke() or .stream(), pass in a config; e.g., chain.invoke({'input': 'foo'}, {'configurable': {'session_id': '[your-value-here]'}})

In [42]:
!curl http://localhost:11434/api/generate


curl: (7) Failed to connect to localhost port 11434 after 0 ms: Connection refused


We have now received an answer for a provided question. We can also view the conversation history and source documents in the response.


In [39]:
print("Conversation Chain")
print(qa1)

Conversation Chain


NameError: name 'qa1' is not defined

### Calculate Similarity score between the LLM Response and context documents

In [ ]:

answer = qa1["answer"]
context_docs = qa1["context"]
similarity_score = calculate_similarity_score(answer, context_docs)

print("Context Similarity Score:", round(similarity_score,2))

Context Similarity Score: 0.75


### Sumamry
This notebook demonstrates the implementation of a Retrieval-Augmented Generation (RAG) pipeline using Llama 3.2 via the Ollama API. We began by preparing and splitting PDF, docx or csv documents into manageable chunks, then ingested these into a vector database with FAISS and Hugging Face embeddings for efficient retrieval.

We integrated the retriever with a conversation chain driven by an LLM, using customized system prompts to generate concise responses based on relevant documents. Additionally, we implemented a method for managing conversation history to maintain context in multi-turn interactions, along with calculating similarity scores between generated answers and source documents using SentenceTransformers.

Overall, this notebook serves as a guide for creating a locally deployable RAG application, effectively combining Llama 3.2 for inference and FAISS for document retrieval.